In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from torch.nn.utils import prune

# 1. DistilBERT 모델과 토크나이저 로딩
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

In [ ]:
# 2. 데이터셋 로딩 (예: AG News dataset)
dataset = load_dataset("ag_news")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
# 3. 데이터셋 전처리 (토크나이징)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
# 4. 전처리된 데이터셋
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# 5. 모델의 일부 레이어에 대해 프루닝 적용
def prune_model(model):
    # DistilBERT의 모든 Linear 레이어에 대해 프루닝을 적용
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name="weight", amount=0.2)  # 20% 프루닝 : 비구조적 푸루닝
    return model

# 모델에 프루닝 적용
model = prune_model(model)

In [ ]:
# 6. Trainer 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# 7. 훈련 시작
trainer.train()

In [ ]:
# 8. 모델 평가
results = trainer.evaluate()

In [ ]:
# 9. 모델을 사용한 예측
texts = ["The quick brown fox jumps over the lazy dog.", "I love machine learning."]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

print("Predictions: ", predictions)

#### 변경 사항
- `prune_model` 함수: 모델의 모든 Linear 레이어에 대해 `l1_unstructured` 프루닝을 적용합니다. 이 방법은 *가중치 값이 작은 뉴런(파라미터)* 을 제거하는 방식으로, *모델 크기를 줄이고 연산 속도를 향상*시킬 수 있습니다.
    - `amount=0.2`: 20%의 가중치를 제거하도록 설정합니다. 이 값을 조정하여 더 많은 또는 적은 가중치를 제거할 수 있습니다.

#### 프루닝 적용 후의 변화
- 모델 크기: 모델의 일부 파라미터가 제거되므로 모델 크기가 줄어듭니다.
- 연산 속도: 프루닝 후에 연산을 최적화할 수 있으며, 특히 추론 시에 성능 향상이 있을 수 있습니다.
- 훈련 시간: 프루닝을 적용한 후 훈련 시간이 일시적으로 줄어들 수 있지만, 성능 손실이 있을 수도 있습니다. 따라서 훈련 후 평가를 통해 성능을 확인해야 합니다.

#### 추가 고려 사항
1. 프루닝 후 fine-tuning: 프루닝 후 모델의 성능이 떨어질 수 있기 때문에, 프루닝을 적용한 후 fine-tuning(추가 훈련)을 진행하여 모델 성능을 회복시키는 것이 좋습니다.
2. 프루닝 비율: `amount` 값을 적절히 조정하여 성능 손실 없이 최적화할 수 있습니다. 너무 많은 프루닝은 모델 성능을 크게 떨어뜨릴 수 있습니다.
3. `Layer-wise Pruning`: 특정 레이어에 대해 프루닝을 다르게 설정할 수도 있습니다. 예를 들어, 상위 레이어는 프루닝을 적게, 하위 레이어는 더 많이 프루닝하는 방식입니다.

#### 요약
- 제공된 코드에 프루닝을 적용하여 모델 크기를 줄이고 연산을 최적화할 수 있습니다.
- 프루닝은 **가중치가 작은 파라미터**를 제거하여 모델을 경량화하고 성능을 최적화합니다.
- **fine-tuning**을 통해 프루닝 후 성능을 다시 개선할 수 있습니다.
